In [ ]:
# login into huggingface to access the data
from huggingface_hub import login
login()

In [ ]:
from wikidata.client import Client
from tqdm import tqdm
import requests
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from datasets import load_dataset
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
!pip install datasets --quiet
!pip install wikidata --quiet


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# loading the given dataset
dataset = load_dataset('sapienzanlp/nlp2025_hw1_cultural_dataset')

In [ ]:
# splitting dataset into train and validation
train = pd.DataFrame(dataset['train'])
validation = pd.DataFrame(dataset['validation'])

### Wikipedia/Wikidata

In [ ]:
# function to request information from wikidata and wikipedia
def count_links(entity_id):
  client = Client()
  item = client.get(entity_id, load=True)
  sitelinks = item.data.get("sitelinks", {})
  count = 0
  for site_key, site_data in sitelinks.items():
      if site_key.endswith("wiki") and not site_key.startswith("commons"):
          lang = site_key.replace("wiki", "")

          # accessing for each entity the english version of wikipedia
          if lang == 'en':
            title = site_data["title"]
            try:
              api_url = "https://en.wikipedia.org/w/api.php"
              params = {
                  "action": "query",
                  "prop": "extracts",
                  "explaintext": True,
                  "titles": title,
                  "format": "json",
                  "redirects": 1
              }

              res = requests.get(api_url, params=params).json()
              page = next(iter(res["query"]["pages"].values()))
              # extracting the whole text
              text = page.get("extract", "")
            except:
              # errors occur from some entities: in this case set text to ''
              text=''

          # count+1 to count in how many languages the article exists
          count+=1

  try:
    # calculate the length of the english version by splitting the text
    length_page = len(text.split())
  except:
    # if errors occur: set length_page to 0
    length_page = 0

  # return our two extracted variables
  return count, length_page

In [ ]:
# initializing lists to save results
counts_validation = []
length_page_validation = []

# applying the above function to each item of the validation set
for i, row in tqdm(validation.iterrows()):
  count, length = count_links(row['item'].strip().split("/")[-1])
  counts_validation.append(count)
  length_page_validation.append(length)

300it [04:04,  1.23it/s]


In [ ]:
# initializing lists to save results
counts_train = []
length_page_train = []

# applying the above function to each item of the train set
for i, row in tqdm(train.iterrows()):
  try:
    count, length = count_links(row['item'].strip().split("/")[-1])
    counts_train.append(count)
    length_page_train.append(length)

  # if error occur set counts and length to 0
  except:
    print("Error in row:", row['item'].strip().split("/")[-1])
    counts_train.append(0)
    length_page_train.append(0)

1063it [13:15,  9.20s/it]

Error in row: Q663109


1356it [17:13, 10.37s/it]

Error in row: Q5164961


1899it [24:37, 10.54s/it]

...

5195it [1:06:23, 12.20s/it]

Error in row: Q24175555


5246it [1:06:59,  1.76it/s]

Error in row: Q7551241


6251it [1:20:28,  1.29it/s]


In [ ]:
# create validation and train datasets consisting of name, label, count and length_page
df_final_validation = pd.DataFrame({'name': validation.name, 'label':validation.label ,'counts': counts_validation, 'length_page': length_page_validation})
df_final_train = pd.DataFrame({'name': train.name, 'label':train.label ,'counts': counts_train, 'length_page': length_page_train})

In [ ]:
# using One-Hot-Encoding for category and subgategory (need for numeric features)
encoder_categories = OneHotEncoder(sparse_output=False)
encoded_array_train = encoder_categories.fit_transform(train[['category']])
encoded_df_train_categories = pd.DataFrame(encoded_array_train, columns=encoder_categories.get_feature_names_out(['category']))

encoded_array_validation = encoder_categories.transform(validation[['category']])
encoded_df_validation_categories = pd.DataFrame(encoded_array_validation, columns=encoder_categories.get_feature_names_out(['category']))


encoder_subcategories = OneHotEncoder(sparse_output=False)
encoded_array_validation = encoder_subcategories.fit_transform(validation[['subcategory']])
encoded_df_validation_subcategories = pd.DataFrame(encoded_array_validation, columns=encoder_subcategories.get_feature_names_out(['subcategory']))

encoded_array_train = encoder_subcategories.transform(train[['subcategory']])
encoded_df_train_subcategories = pd.DataFrame(encoded_array_train, columns=encoder_subcategories.get_feature_names_out(['subcategory']))

In [ ]:
# adding the created One-Hot-Encodings to the final datasets (train and validation)
df_final_validation = pd.concat([df_final_validation, encoded_df_validation_categories, encoded_df_validation_subcategories], axis=1)
df_final_train = pd.concat([df_final_train, encoded_df_train_categories, encoded_df_train_subcategories], axis=1)

### Word embeddings

In [ ]:
!curl -L -O http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   308    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   346    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  822M  100  822M    0     0  2246k      0  0:06:14  0:06:14 --:--:-- 2502k47  0:00:08  0:35:39  519k:59  0:00:19  0:15:40 1831k0  0:14:58  0:00:22  0:14:36 1375k   0     0  1008k      0  0:13:55  0:00:26  0:13:29 1400k  0     0  1309k      0  0:10:42  0:00:33  0:10:09 2745k2210k0  0:06:30 2203k07:54  0:02:02  0:05:52 2260k 0:02:36  0:04:48 2911k   0     0  1943k      0  0:07:13  0:02:44  0:04:29 2906k13  0:03:34  0:03:39 1720k0:07:12  0:03:50  0:03:22 2308k1951k      0  0:07:11  0:03:55  0:03:16 2164k:03:57  0:03:13 2215k  0:01:39 2581k 2117k15  0:06:06  0:00:09 2417k    0  0:06:15  0:06:12  0:00:03 2347k
Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  infl

In [ ]:
# loading glove embedding
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            vector = np.array(parts[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# using 100d version
glove_path = "glove.6B.100d.txt"
glove_embeddings = load_glove_embeddings(glove_path)

In [ ]:
# getting glove embedding for each item
def get_item_vector(item, embeddings, dim=100):
    item = item.lower()
    # if item consists of more than one word
    if len(item.split())>1:
      # splitting it into different words
      words = item.lower().split()
      # computing embedding for each word
      valid_vectors = [embeddings[w] for w in words if w in embeddings]

      if valid_vectors:
          # computing the mean for each item
          return np.mean(valid_vectors, axis=0)
      else:
          # fallback if none of the words are found: setting embeddings to zero
          return np.zeros(dim)

    # if item consists of one word: using embedding of this word
    else:
      return embeddings[item]

In [ ]:
# initializing lists to save results
train_vectors = []
validation_vectors = []

# applying the above function to each item of the train set
for i, row in tqdm(df_final_train.iterrows()):
  try:
    vector = get_item_vector(row['name'], glove_embeddings)

  # implementing the fallback if item consists of one word
  except:
    vector = 100*[0]
  train_vectors.append(vector)

# applying the above function to each item of the validation set
for i, row in tqdm(df_final_validation.iterrows()):
  try:
    vector = get_item_vector(row['name'], glove_embeddings)
  # implementing the fallback if item consists of one word
  except:
    vector = 100*[0]
  validation_vectors.append(vector)

6251it [00:00, 31604.58it/s]
300it [00:00, 15685.31it/s]


In [ ]:
# creating data frame consisting of embeddings of train set
train_vectors = np.array(train_vectors)
for i in range(train_vectors.shape[1]):
    df_final_train[f"glove_{i}"] = train_vectors[:, i]

# creating data frame consisting of embeddings of validation set
validation_vectors = np.array(validation_vectors)
for i in range(validation_vectors.shape[1]):
    df_final_validation[f"glove_{i}"] = validation_vectors[:, i]

/var/folders/fc/yts100z16_7_drgqty0jrh440000gn/T/ipykernel_31055/1476810809.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_train[f"glove_{i}"] = train_vectors[:, i]
/var/folders/fc/yts100z16_7_drgqty0jrh440000gn/T/ipykernel_31055/1476810809.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_train[f"glove_{i}"] = train_vectors[:, i]
/var/folders/fc/yts100z16_7_drgqty0jrh440000gn/T/ipykernel_31055/1476810809.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

In [ ]:
# saving the datasets
df_final_validation.to_csv('validation_vectors.csv', index=False)
df_final_train.to_csv('train_vectors.csv', index=False)